In [1]:
import json
import requests
import hashlib
import hmac
import time
from datetime import datetime
from urllib.parse import urlencode

class CryptoTradingSim:
    def __init__(self, keys_file="keys.json"):
        self.keys_file = keys_file
        self.trade_history = []
        self.transfer_history = []
        self.api_endpoints = {
            "binance": "https://testnet.binance.vision",  # Using Binance Testnet
            "bybit": "https://api-testnet.bybit.com"  # Using Bybit Testnet
        }
        self.api_keys = self.load_api_keys()

    def load_api_keys(self):
        """Load API keys from keys.json file."""
        try:
            with open(self.keys_file, "r") as file:
                return json.load(file)
        except FileNotFoundError:
            return {}

    def get_price(self, exchange_name, coin_name):
        """Fetches the price of a cryptocurrency from the specified exchange."""
        coin_symbol = coin_name.upper() + "USDT"
        print(f"Fetching price for {coin_symbol} on {exchange_name}...")
        
        if exchange_name.lower() == "binance":
            url = f"{self.api_endpoints['binance']}/api/v3/ticker/price?symbol={coin_symbol}"
        elif exchange_name.lower() == "bybit":
            url = f"{self.api_endpoints['bybit']}/spot/v1/ticker?symbol={coin_symbol}"
        else:
            print("Exchange not supported")
            return "Exchange not supported"
        
        try:
            response = requests.get(url)
            data = response.json()
            print("API Response:", data)
            if exchange_name.lower() == "binance":
                price = float(data["price"])
            elif exchange_name.lower() == "bybit":
                price = float(data["result"]["price"])
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            return {"exchange": exchange_name, "coin": coin_name, "price": price, "timestamp": timestamp}
        except Exception as e:
            print("Error fetching price:", str(e))
            return {"error": str(e)}

    def create_signature(self, params, secret):
        """Generates an HMAC SHA256 signature for Binance or Bybit API requests."""
        query_string = urlencode(params)
        signature = hmac.new(secret.encode(), query_string.encode(), hashlib.sha256).hexdigest()
        print(f"Generated Signature: {signature}")
        return signature
    
    def get_balance(self, exchange_name):
        """Fetches account balance from the specified exchange."""
        print(f"Fetching balance for {exchange_name}...")
        api_key = self.api_keys.get(exchange_name, {}).get("apiKey")
        secret_key = self.api_keys.get(exchange_name, {}).get("secret")
        if not api_key or not secret_key:
            print("Missing API keys")
            return f"Missing {exchange_name.capitalize()} API keys"
        
        if exchange_name == "binance":
            endpoint = f"{self.api_endpoints['binance']}/api/v3/account"
            params = {"timestamp": int(time.time() * 1000)}
            params["signature"] = self.create_signature(params, secret_key)
            headers = {"X-MBX-APIKEY": api_key}
            response = requests.get(endpoint, headers=headers, params=params)
        elif exchange_name == "bybit":
            endpoint = f"{self.api_endpoints['bybit']}/spot/v1/account"
            params = {"api_key": api_key, "timestamp": int(time.time() * 1000)}
            params["sign"] = self.create_signature(params, secret_key)
            response = requests.get(endpoint, params=params)
        else:
            print("Exchange not supported")
            return "Exchange not supported"
        
        try:
            response_json = response.json()
            print("Balance Response:", response_json)
            return response_json
        except Exception as e:
            print("Error fetching balance:", str(e))
            return {"error": str(e)}

    def get_lot_size(self, symbol):
        """Fetches Binance trading rules and returns the correct lot size step for a symbol."""
        print(f"Fetching lot size for {symbol}...")
        url = f"{self.api_endpoints['binance']}/api/v3/exchangeInfo"
        response = requests.get(url)
        data = response.json()
        print("Exchange Info Response:", data)
        
        for s in data["symbols"]:
            if s["symbol"] == symbol:
                for f in s["filters"]:
                    if f["filterType"] == "LOT_SIZE":
                        lot_size = float(f["stepSize"])
                        print(f"Lot size for {symbol}: {lot_size}")
                        return lot_size
        return None
    
    def round_quantity(self, symbol, quantity):
        """Rounds quantity based on Binance's lot size requirements."""
        step_size = self.get_lot_size(symbol)
        if step_size:
            precision = str(step_size)[::-1].find('.')  # Find decimal places required
            rounded_quantity = round(quantity - (quantity % step_size), precision)
            print(f"Rounded Quantity for {symbol}: {rounded_quantity}")
            return rounded_quantity
        print(f"No step size found, returning original quantity: {quantity}")
        return quantity

    def place_order(self, exchange, side, coin_name, amount):
        """Places a test order on Binance or Bybit Testnet with correct lot sizing."""
        print(f"Placing {side} order for {amount} {coin_name} on {exchange}...")
        api_key = self.api_keys.get(exchange, {}).get("apiKey")
        secret_key = self.api_keys.get(exchange, {}).get("secret")
        if not api_key or not secret_key:
            print("Missing API keys")
            return f"Missing {exchange.capitalize()} API keys"
        
        symbol = coin_name.upper() + "USDT"
        amount = self.round_quantity(symbol, amount)
        
        params = {
            "symbol": symbol,
            "side": side.upper(),
            "type": "MARKET",
            "quantity": amount,
            "timestamp": int(time.time() * 1000)
        }
        print("Order Params:", params)
        
        if exchange == "binance":
            params["signature"] = self.create_signature(params, secret_key)
            headers = {"X-MBX-APIKEY": api_key}
            endpoint = f"{self.api_endpoints['binance']}/api/v3/order/test"  # Test order
            response = requests.post(endpoint, headers=headers, params=params)
        elif exchange == "bybit":
            params["api_key"] = api_key
            params["sign"] = self.create_signature(params, secret_key)
            headers = {}
            endpoint = f"{self.api_endpoints['bybit']}/spot/v1/order"
            response = requests.post(endpoint, headers=headers, data=params)
        else:
            print("Exchange not supported")
            return "Exchange not supported"
        
        try:
            response_json = response.json()
            print("Order Response:", response_json)
            return response_json
        except Exception as e:
            print("Error placing order:", str(e))
            return {"error": str(e)}
    
    def buy(self, exchange_name, coin_name, amount_in_USD, paper_trade=True):
        """Executes a test buy order on Binance or Bybit Testnet."""
        print(f"Executing BUY order on {exchange_name} for {amount_in_USD} USD in {coin_name}...")
        if exchange_name.lower() not in ["binance", "bybit"]:
            return "Only Binance Testnet and Bybit Testnet supported for real trades."
        
        if paper_trade:
            print("Paper trading mode: No real order placed.")
            return "Paper trading enabled. No real order placed."
        
        return self.place_order(exchange_name.lower(), "BUY", coin_name, amount_in_USD)
    
    def sell(self, exchange_name, coin_name, amount_in_USD, paper_trade=True):
        """Executes a test sell order on Binance or Bybit Testnet."""
        print(f"Executing SELL order on {exchange_name} for {amount_in_USD} USD in {coin_name}...")
        if exchange_name.lower() not in ["binance", "bybit"]:
            return "Only Binance Testnet and Bybit Testnet supported for real trades."
        
        if paper_trade:
            print("Paper trading mode: No real order placed.")
            return "Paper trading enabled. No real order placed."
        
        return self.place_order(exchange_name.lower(), "SELL", coin_name, amount_in_USD)

In [ ]:

# Initialize the simulator
sim = CryptoTradingSim(keys_file="testnet_keys.json")
binance_balance = sim.get_balance("binance")
with open("binance_balance.json", "w") as file:
    json.dump(binance_balance, file, indent=4)
buy_order = sim.buy("binance", "BTC", 100, paper_trade=False)  # Buys $100 worth of BTC on Binance Testnet
print(buy_order)
sim.get_balance("bybit")
print(buy_order)


Fetching balance for binance...
Generated Signature: 22900dbe3ccd4959ff029082e1c3bbd7a13b35384bdf94336b56ed5bd576560a
Balance Response: {'makerCommission': 0, 'takerCommission': 0, 'buyerCommission': 0, 'sellerCommission': 0, 'commissionRates': {'maker': '0.00000000', 'taker': '0.00000000', 'buyer': '0.00000000', 'seller': '0.00000000'}, 'canTrade': True, 'canWithdraw': True, 'canDeposit': True, 'brokered': False, 'requireSelfTradePrevention': False, 'preventSor': False, 'updateTime': 1741757652415, 'accountType': 'SPOT', 'balances': [{'asset': 'ETH', 'free': '1.00000000', 'locked': '0.00000000'}, {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'}, {'asset': 'LTC', 'free': '4.00000000', 'locked': '0.00000000'}, {'asset': 'BNB', 'free': '1.00000000', 'locked': '0.00000000'}, {'asset': 'USDT', 'free': '10000.00000000', 'locked': '0.00000000'}, {'asset': 'TRX', 'free': '2059.00000000', 'locked': '0.00000000'}, {'asset': 'XRP', 'free': '201.00000000', 'locked': '0.00000000'}, {